### MY HOUSE BUILDER

Come sempre ho iniziato il workshop prendendo l'immagine di riferimento (qui sotto)

![img](./planimetria.jpg)

La cosa più impegnativa di questo homework è stata senza dubbio la creazione del svg adeguato che rappresentasse l'immagine di riferimento.
Ho disegnato l'svg ricalcando la struttura su Graphics un software per il disegno vettoriale. Esportati il livello per le mura interne, le mura esterne, le porte e le finestre, ho passato questi file in formato lines e li ho posizionati nella omonima cartella. La funzione generate_structure richiama generate_substructure per creare le struttre per ogni livello leggendo dai file lines. A questo punto di estrudono i muri esterni in maniera da farli essere più spessi di quelli interni. si moltiplica per l'altezza dei muri e ad i muri esterno si sottraggono le porte e le finestre con DIFF. Alle mura interne si sottraggono solo le porte. Infine si applicano le texture a muri interni esterni ed al pavimento creato sulla base di quanto occupava la struttura. Nelle altre directory è possible trovare l'svg completo e altri modelli.


In [4]:
import csv
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.004816 seconds


In [7]:
"""generate_structure

Il seguente metodo crea il modello di casa contenuto nella directory indicata.
Tale modello deve essere composto da quattro file lines: door, windows, external_wall, wall

@param directory: di default lines, nome della cartella dove sono contentenuti i file lines
@param dx,dy,dz: l'immagine viene scalata sulla base di questi valori, di deafult sono ad 1
@return: hpc della struttura
"""
def generate_structure(directory="lines",dx=1,dy=1,dz=1):
    doors=OFFSET([10,10])(generate_sub_structure("./"+directory+"/door.lines"))
    doors=PROD([doors,QUOTE([25])])
    
    windows=OFFSET([10,10])(generate_sub_structure("./"+directory+"/windows.lines"))
    windows=PROD([windows,QUOTE([15])])
    windows=STRUCT([T(3)(10),windows])
    
    walls=generate_sub_structure("./"+directory+"/external_wall.lines")
    floor=SOLIDIFY(walls)
    floor = TEXTURE(["./pattern/floor.jpg", TRUE, TRUE, 5, 5, 0, 5, 5])(floor)
    walls=(OFFSET([5,5])(walls))
    walls=PROD([walls,QUOTE([35])])
    internal_walls=OFFSET([2,2])(generate_sub_structure("./"+directory+"/wall.lines"))
    internal_walls=PROD([internal_walls,QUOTE([35])])
    internal_walls=DIFF([internal_walls,doors])
    internal_walls = TEXTURE(["./pattern/wooden.jpg", TRUE, FALSE, 1, 1, 0, 1, 1])(internal_walls)
    walls=DIFF([walls,windows])
    walls=DIFF([walls,doors])
    walls = TEXTURE(["./pattern/brick.jpg", TRUE, TRUE, 5, 5, 0, 5, 5])(walls)
    building=STRUCT([(internal_walls),(walls)])
    building=STRUCT([floor,building])
    building=STRUCT([R([1,2])(math.pi/6),(building)])
    building=STRUCT([S([1,2,3])([dx,dy,dz]),building])
    
    return building

"""generate_sub_structure

Il seguente metodo crea l'hpc di una sottostruttura descritta da un file con formato lines.
Nello specifico viene fatta una semplice lettua a due e due si aggiungono vertici e faccette che veranno 
utilizzate per la creazione della polyline con MKPOL.

@param file_name: nome del file in formato .lines
@return: hpc della struttura
"""
def generate_sub_structure(file_name):
    i=0
    points=[]
    indesxs=[]
    with open(file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            points.append([float(row[0]),float(row[1])])
            points.append([float(row[2]),float(row[3])])
            i=i+2
            indesxs.append([i-1,i])
    return MKPOL([points,indesxs,None])
                

In [8]:
VIEW(generate_structure())


<function SOLIDIFY at 0x10e1d5500>


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1050970f0> >

### RISULTATI
Strutture intermedie

![prova ](./img/frame1.png)

![prova ](./img/frame2.png)

Strutture con texture 

![prova ](./img/frame3.png)

![prova ](./img/frame4.png)